In [3]:
import json  
  
def read_jsonl(file_path):  
    """  
    读取 JSONL 文件中的数据并返回一个包含所有记录的字典列表。  
  
    参数:  
    file_path (str): JSONL 文件的路径。  
  
    返回:  
    list: 包含所有记录的字典列表。  
    """  
    data = []  
  
    with open(file_path, 'r', encoding='utf-8') as file:  
        for line in file:  
            # 解析每一行的 JSON 对象并添加到列表中  
            data.append(json.loads(line.strip()))  
  
    return data  

In [6]:
import random
import json  
import pandas as pd  
  
# 读取JSON文件  
ana = "data/annotation/Meta-Llama-3.1-8B-Instruct_sample.json"  
with open(ana, "r") as f:  
    data = json.load(f)  
  
# 创建字典  
# judge_dict = {item['question_id']: [item['query']] for item in data}  
question = [item["question_id"] for item in data]

# 创建DataFrame  
df = pd.DataFrame(question, columns=["question_id"])  
  
# 保存到CSV文件  
# df.to_csv("data/sample_query/sample_query.csv", index=False)  
random_indices = question
# random_indices = [item["question_id"] for item in question_id]
def get_judge_answer(model_name):
    # data = read_jsonl("data/alpaca/model_judgment/gpt-4o_images/tulu_v2_8b_2048_default_template_dpo.jsonl")
    data = read_jsonl(f"data/arena-ta/model_judgment/gpt-4o_images/{model_name}.jsonl")
    model_answer1 = read_jsonl("data/arena-ta/model_answer/gpt-4-1106-preview.jsonl") # baseline answer
    model_answer2 = read_jsonl(f"data/arena-ta/model_answer/{model_name}.jsonl")
    question = read_jsonl(f"data/arena-ta/question.jsonl")
    question_dict = {item["question_id"]: {"cluster":item["cluster"], "query":item["turns"][0]["content"]} for item in question}
    # data = read_jsonl("data/alpaca/model_judgment/gpt-4o/Meta-Llama-3.1-8B-Instruct.jsonl")
    # data = read_jsonl("data/arena-hard-v0.1/model_judgment/gpt-4o/Meta-Llama-3.1-8B-Instruct.jsonl")
    score_A = [item['games'][0]['score'] for item in data]
    score_B = [item['games'][1]['score'] for item in data]
    score_all = [[x1, x2] for x1, x2 in zip(score_A, score_B)]
    score_all
    score_map = {
        "A>>B": 5,
        "A>B": 4,
        "A=B": 3,
        "B>A": 2,
        "B>>A": 1,
        None:0
    }
    score_map_generall = {
        "A>>B": 3,
        "A>B": 3,
        "A=B": 2,
        "B>A": 1,
        "B>>A": 1,
        None:0
    }
    score_all_quant = [[score_map[x1], score_map[x2]] for x1, x2 in score_all]
    score_generall = [[score_map_generall[x1], score_map_generall[x2]] for x1, x2 in score_all]
    null_judges = []
    for i, (x1, x2) in enumerate(score_all_quant):
        if x1 == 0 or x2 == 0:
            null_judges.append(i)
    print(f"null_judges: {len(null_judges)}")
    same_judges = []
    different_judges = []
    for i, (x1, x2) in enumerate(score_generall):
        if (x1 == 1 and x2 == 3) or (x1 == 3 and x2 == 1) or (x1 == 2 and x2 == 2):
            same_judges.append(i)
        if (x1 == x2 and x1 != 2):
            different_judges.append(i)
    print(f"same_judges: {len(same_judges)}")
    print(f"different_judges: {len(different_judges)}")
    good_samples = []
    for i, (x1, x2) in enumerate(score_generall):
        if x1 == 1 and x2 == 3:
            good_samples.append(i)
    print(f"good_samples: {len(good_samples)}")
    judge_dict = {item['question_id']:[item['games'][0]['score'], item['games'][0]['judgment'], item['games'][1]['score'], item['games'][1]['judgment']]  for item in data}
    model_answer1_dict = {item['question_id']:[item['choices'][0]['turns'][0]['content'], item['choices'][0]['turns'][0]['token_len'], item['model_id']]  for item in model_answer1}
    model_answer2_dict = {item['question_id']:[item['choices'][0]['turns'][0]['content'], item['choices'][0]['turns'][0]['token_len'], item['model_id']]  for item in model_answer2}
    res = []
    index = 0
    # for index, idx in enumerate(good_samples):
    # for index, idx in enumerate(good_samples):
    # random_indices = random.sample(range(len(data)), 50)  

  
    for index, idx in enumerate(random_indices):
        question_id = idx
        # if score != judge_dict[question_id][0]:
        res.append({
            "index": index,
            "question_id":question_id, 
            "query": question_dict[question_id]["query"],
            "cluster": question_dict[question_id]["cluster"],
            "score_1":judge_dict[question_id][0], 
            "score_2":judge_dict[question_id][2], 
            "judgment_1":judge_dict[question_id][1], 
            "judgment_2":judge_dict[question_id][3],
            "model_answer1": model_answer1_dict[question_id][0],
            "token_len1": model_answer1_dict[question_id][1],
            "model_id_1":model_answer1_dict[question_id][2],
            "model_answer2": model_answer2_dict[question_id][0],
            "token_len2": model_answer2_dict[question_id][1],
            "model_id_2":model_answer2_dict[question_id][2],
            })
    return res
# model_name = "tulu_v2_8b_default_template_dpo_list_bsz1_trible_debug"
# res = get_judge_answer(model_name)
# Model list  
# model_list = ["ta_llama3_instruct_dpo_list_bsz1_trible_debug_v2-1500", "Meta-Llama-3.1-8B-Instruct", "Meta-Llama-3.1-70B-Instruct", "tulu-2-dpo-70b", "Qwen2-72B-Instruct"]  # Replace with actual model names  

model_list = ["Meta-Llama-3.1-8B-Instruct", "Meta-Llama-3.1-70B-Instruct", "tulu-2-dpo-70b", "Qwen2-72B-Instruct"]  # Replace with actual model names  
# Loop through each model and get 50 random samples  
for model_name in model_list:  
    res = get_judge_answer(model_name)  
    print(f"model_name: {model_name}, len: {len(res)}")
    # random_samples = random.sample(res, 50) 
    with open(f"data/annotation_2/{model_name}_sample.json", 'w') as f:
        json.dump(res, f, indent=4)  
    # generate_res(res, model_name) 


null_judges: 1
same_judges: 284
different_judges: 152
good_samples: 45
model_name: Meta-Llama-3.1-8B-Instruct, len: 50
null_judges: 0
same_judges: 287
different_judges: 139
good_samples: 55
model_name: Meta-Llama-3.1-70B-Instruct, len: 50
null_judges: 0
same_judges: 388
different_judges: 93
good_samples: 24
model_name: tulu-2-dpo-70b, len: 50
null_judges: 0
same_judges: 398
different_judges: 79
good_samples: 36
model_name: Qwen2-72B-Instruct, len: 50


In [9]:
import json  
import os  
import pandas as pd  
  
def calculate(data):  
    good_idx = []  
    same_idx = []  
    loss_idx = []  
    result_label = []  
    for idx, item in enumerate(data[:50]):  
        score_map_generall = {  
            "A>>B": 3,  
            "A>B": 3,  
            "A=B": 2,  
            "B>A": 1,  
            "B>>A": 1,  
            None: 0  
        }  
        score_map_generall2 = {  
            "A>>B": 1,  
            "A>B": 1,  
            "A=B": 2,  
            "B>A": 3,  
            "B>>A": 3,  
            None: 0  
        }  
        score_1 = score_map_generall[item["score_1"]]  
        score_2 = score_map_generall2[item["score_2"]]  
        if score_1 != score_2:  
            same_idx.append(idx)  
            result_label.append("T")  
        elif score_1 == 3:  
            good_idx.append(idx)  
            result_label.append("W")  
        elif score_2 == 1:  
            loss_idx.append(idx)  
            result_label.append("L")  
        else:  
            same_idx.append(idx)  
            result_label.append("T")  
    result = {  
        "good_idx": len(good_idx),  
        "same_idx": len(same_idx),  
        "loss_idx": len(loss_idx),  
        "sum": len(good_idx) + len(same_idx) + len(loss_idx)  
    }  
    return result, result_label  
  
def cal_text(data):  
    model_name = data[0]["model_id_2"]  
    judge_data = read_jsonl(f"data/arena-ta/model_judgment/gpt-4o_text/{model_name}.jsonl")  
    judge_dict = {item['question_id']: [item['games'][0]['score'], item['games'][0]['judgment'], item['games'][1]['score'], item['games'][1]['judgment']] for item in judge_data}  
    res = []  
    result_label = []  
    for idx, item in enumerate(data):  
        score_1 = judge_dict[item["question_id"]][0]  
        judge_1 = judge_dict[item["question_id"]][1]  
        score_2 = judge_dict[item["question_id"]][2]  
        judge_2 = judge_dict[item["question_id"]][3]  
        item["text_score1"] = score_1  
        item["text_judge1"] = judge_1  
        item["text_score2"] = score_2  
        item["text_judge2"] = judge_2  
        good_idx = []  
        same_idx = []  
        loss_idx = []  
        res.append(item)  
    for idx, item in enumerate(res[:50]):  
        score_map_generall = {  
            "A>>B": 3,  
            "A>B": 3,  
            "A=B": 2,  
            "B>A": 1,  
            "B>>A": 1,  
            None: 0  
        }  
        score_map_generall2 = {  
            "A>>B": 1,  
            "A>B": 1,  
            "A=B": 2,  
            "B>A": 3,  
            "B>>A": 3,  
            None: 0  
        }  
        score_1 = score_map_generall[item["text_score1"]]  
        score_2 = score_map_generall2[item["text_score2"]]  
        if score_1 != score_2:  
            same_idx.append(idx)  
            result_label.append("T")  
        elif score_1 == 3:  
            good_idx.append(idx)  
            result_label.append("W")  
        elif score_2 == 1:  
            loss_idx.append(idx)  
            result_label.append("L")  
        else:  
            same_idx.append(idx)  
            result_label.append("T")  
    result = {  
        "good_idx": len(good_idx),  
        "same_idx": len(same_idx),  
        "loss_idx": len(loss_idx),  
        "sum": len(good_idx) + len(same_idx) + len(loss_idx)  
    }  
    return result, result_label  
  
def read_jsonl(file_path):  
    with open(file_path, 'r') as file:  
        return [json.loads(line) for line in file]  
  
def merge_results(model_name, image_res, text_res):  
    merged_result = {  
        "model": model_name,  # 添加模型名称到结果字典的第一项  
        "good_idx_image": image_res["good_idx"],  
        "same_idx_image": image_res["same_idx"],  
        "loss_idx_image": image_res["loss_idx"],  
        "sum_image": image_res["sum"],  
        "good_idx_text": text_res["good_idx"],  
        "same_idx_text": text_res["same_idx"],  
        "loss_idx_text": text_res["loss_idx"],  
        "sum_text": text_res["sum"]  
    }  
    return merged_result  
  
# 获取data/annotation/目录下的所有文件  
annotation_dir = 'data/annotation_2/'  
files = [f for f in os.listdir(annotation_dir) if os.path.isfile(os.path.join(annotation_dir, f))]  
  
merged_results = []  
label_results_text = {}  
label_results_image = {}  
  
# 遍历每个文件并进行calculate统计  
for file_name in files:  
    file_path = os.path.join(annotation_dir, file_name)  
    with open(file_path, 'r') as f:  
        data = json.load(f)  
    print(f"Model: {file_name}")  
    res, result_label_image = calculate(data)  
    res_text, result_label_text = cal_text(data)  
    merged_res = merge_results(file_name, res, res_text)  # 传递模型名称  
    merged_results.append(merged_res)  
    model_name = file_name.split("_")[0]  
    label_results_text[model_name] = result_label_text  
    label_results_image[model_name] = result_label_image  
  
# 指定的模型顺序  
model_order = ["Meta-Llama-3.1-8B-Instruct", "Meta-Llama-3.1-70B-Instruct", "Qwen2-72B-Instruct", "tulu-2-dpo-70b"]  
  
# 按指定顺序排序 merged_results  
ordered_merged_results = sorted(merged_results, key=lambda x: model_order.index(x["model"]) if x["model"] in model_order else float('inf'))  
  
# 创建一个DataFrame并保存为CSV文件  
df = pd.DataFrame(ordered_merged_results)  
  
# 确保模型名称是第一列  
df = df[["model", "good_idx_image", "same_idx_image", "loss_idx_image", "sum_image", "good_idx_text", "same_idx_text", "loss_idx_text", "sum_text"]]  
  
df.to_csv("merged_results.csv", index=False)  
print("Merged results saved to merged_results.csv")  
  
# 保存 label_results_text 为 CSV 文件  
label_text_df = pd.DataFrame.from_dict(label_results_text, orient='index').transpose()  
label_text_df = label_text_df[model_order]  
label_text_df.to_csv("label_results_text_70B.csv", index=False)  
print("Label results text saved to label_results_text.csv")  
  
# 保存 label_results_image 为 CSV 文件  
label_image_df = pd.DataFrame.from_dict(label_results_image, orient='index').transpose()  
label_image_df = label_image_df[model_order]  
label_image_df.to_csv("label_results_image_70B.csv", index=False)  
print("Label results image saved to label_results_image.csv")  


Model: Qwen2-72B-Instruct_sample.json
Model: Meta-Llama-3.1-8B-Instruct_sample.json
Model: Meta-Llama-3.1-70B-Instruct_sample.json
Model: tulu-2-dpo-70b_sample.json
Merged results saved to merged_results.csv
Label results text saved to label_results_text.csv
Label results image saved to label_results_image.csv


In [10]:
import json  
import os  
import pandas as pd  
  
def calculate(data):  
    good_idx = []  
    same_idx = []  
    loss_idx = []  
    result_label = []  
    for idx, item in enumerate(data[:50]):  
        score_map_generall = {  
            "A>>B": 3,  
            "A>B": 3,  
            "A=B": 2,  
            "B>A": 1,  
            "B>>A": 1,  
            None: 0  
        }  
        score_map_generall2 = {  
            "A>>B": 1,  
            "A>B": 1,  
            "A=B": 2,  
            "B>A": 3,  
            "B>>A": 3,  
            None: 0  
        }  
        score_1 = score_map_generall[item["score_1"]]  
        score_2 = score_map_generall2[item["score_2"]]  
        if score_1 != score_2: 
            if score_1 == 3 and score_2 == 2:
                good_idx.append(idx)
                result_label.append("W") 
            elif score_1 == 1 and score_2 == 2:
                loss_idx.append(idx)
                result_label.append("L")
            elif score_1 == 2 and score_2 == 3:
                good_idx.append(idx)
                result_label.append("W")
            elif score_1 == 2 and score_2 == 1:
                loss_idx.append(idx)
                result_label.append("L")
            else:
                same_idx.append(idx)  
                result_label.append("T")  
        elif score_1 == 3:  
            good_idx.append(idx)  
            result_label.append("W")  
        elif score_1 == 1:  
            loss_idx.append(idx)  
            result_label.append("L")  
        else:  
            same_idx.append(idx)  
            result_label.append("T")   
    result = {  
        "good_idx": len(good_idx),  
        "same_idx": len(same_idx),  
        "loss_idx": len(loss_idx),  
        "sum": len(good_idx) + len(same_idx) + len(loss_idx)  
    }  
    return result, result_label  
  
def cal_text(data):  
    model_name = data[0]["model_id_2"]  
    judge_data = read_jsonl(f"data/arena-ta/model_judgment/gpt-4o_text/{model_name}.jsonl")  
    judge_dict = {item['question_id']: [item['games'][0]['score'], item['games'][0]['judgment'], item['games'][1]['score'], item['games'][1]['judgment']] for item in judge_data}  
    res = []  
    result_label = []  
    for idx, item in enumerate(data):  
        score_1 = judge_dict[item["question_id"]][0]  
        judge_1 = judge_dict[item["question_id"]][1]  
        score_2 = judge_dict[item["question_id"]][2]  
        judge_2 = judge_dict[item["question_id"]][3]  
        item["text_score1"] = score_1  
        item["text_judge1"] = judge_1  
        item["text_score2"] = score_2  
        item["text_judge2"] = judge_2  
        good_idx = []  
        same_idx = []  
        loss_idx = []  
        res.append(item)  
    for idx, item in enumerate(res[:50]):  
        score_map_generall = {  
            "A>>B": 3,  
            "A>B": 3,  
            "A=B": 2,  
            "B>A": 1,  
            "B>>A": 1,  
            None: 0  
        }  
        score_map_generall2 = {  
            "A>>B": 1,  
            "A>B": 1,  
            "A=B": 2,  
            "B>A": 3,  
            "B>>A": 3,  
            None: 0  
        }  
        score_1 = score_map_generall[item["text_score1"]]  
        score_2 = score_map_generall2[item["text_score2"]]  
        if score_1 != score_2: 
            if score_1 == 3 and score_2 == 2:
                good_idx.append(idx)
                result_label.append("W") 
            elif score_1 == 1 and score_2 == 2:
                loss_idx.append(idx)
                result_label.append("L")
            elif score_1 == 2 and score_2 == 3:
                good_idx.append(idx)
                result_label.append("W")
            elif score_1 == 2 and score_2 == 1:
                loss_idx.append(idx)
                result_label.append("L")
            else:
                same_idx.append(idx)  
                result_label.append("T")  
        elif score_1 == 3:  
            good_idx.append(idx)  
            result_label.append("W")  
        elif score_1 == 1:  
            loss_idx.append(idx)  
            result_label.append("L")  
        else:  
            same_idx.append(idx)  
            result_label.append("T")  
    result = {  
        "good_idx": len(good_idx),  
        "same_idx": len(same_idx),  
        "loss_idx": len(loss_idx),  
        "sum": len(good_idx) + len(same_idx) + len(loss_idx)  
    }  
    return result, result_label  
  
def read_jsonl(file_path):  
    with open(file_path, 'r') as file:  
        return [json.loads(line) for line in file]  
  
def merge_results(model_name, image_res, text_res):  
    merged_result = {  
        "model": model_name,  # 添加模型名称到结果字典的第一项  
        "good_idx_image": image_res["good_idx"],  
        "same_idx_image": image_res["same_idx"],  
        "loss_idx_image": image_res["loss_idx"],  
        "sum_image": image_res["sum"],  
        "good_idx_text": text_res["good_idx"],  
        "same_idx_text": text_res["same_idx"],  
        "loss_idx_text": text_res["loss_idx"],  
        "sum_text": text_res["sum"]  
    }  
    return merged_result  
  
# 获取data/annotation/目录下的所有文件  
annotation_dir = 'data/annotation_2/'  
files = [f for f in os.listdir(annotation_dir) if os.path.isfile(os.path.join(annotation_dir, f))]  
  
merged_results = []  
label_results_text = {}  
label_results_image = {}  
  
# 遍历每个文件并进行calculate统计  
for file_name in files:  
    file_path = os.path.join(annotation_dir, file_name)  
    with open(file_path, 'r') as f:  
        data = json.load(f)  
    print(f"Model: {file_name}")  
    res, result_label_image = calculate(data)  
    res_text, result_label_text = cal_text(data)  
    merged_res = merge_results(file_name, res, res_text)  # 传递模型名称  
    merged_results.append(merged_res)  
    model_name = file_name.split("_")[0]  
    label_results_text[model_name] = result_label_text  
    label_results_image[model_name] = result_label_image  
  
# 指定的模型顺序  
model_order = ["Meta-Llama-3.1-8B-Instruct", "Meta-Llama-3.1-70B-Instruct", "Qwen2-72B-Instruct", "tulu-2-dpo-70b"]  
  
# 按指定顺序排序 merged_results  
ordered_merged_results = sorted(merged_results, key=lambda x: model_order.index(x["model"]) if x["model"] in model_order else float('inf'))  
  
# 创建一个DataFrame并保存为CSV文件  
df = pd.DataFrame(ordered_merged_results)  
  
# 确保模型名称是第一列  
df = df[["model", "good_idx_image", "same_idx_image", "loss_idx_image", "sum_image", "good_idx_text", "same_idx_text", "loss_idx_text", "sum_text"]]  
  
df.to_csv("merged_results2.csv", index=False)  
print("Merged results saved to merged_results.csv")  
  
# 保存 label_results_text 为 CSV 文件  
label_text_df = pd.DataFrame.from_dict(label_results_text, orient='index').transpose()  
label_text_df = label_text_df[model_order]  
label_text_df.to_csv("label_results_text_70B2.csv", index=False)  
print("Label results text saved to label_results_text.csv")  
  
# 保存 label_results_image 为 CSV 文件  
label_image_df = pd.DataFrame.from_dict(label_results_image, orient='index').transpose()  
label_image_df = label_image_df[model_order]  
label_image_df.to_csv("label_results_image_70B2.csv", index=False)  
print("Label results image saved to label_results_image.csv")  


Model: Qwen2-72B-Instruct_sample.json
Model: Meta-Llama-3.1-8B-Instruct_sample.json
Model: Meta-Llama-3.1-70B-Instruct_sample.json
Model: tulu-2-dpo-70b_sample.json
Merged results saved to merged_results.csv
Label results text saved to label_results_text.csv
Label results image saved to label_results_image.csv
